In [2]:
!apt-get install default-jre
!java -version
!pip install h2o


Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
     |████████████████████████████████| 164.6MB 76kB/s 
  Created wheel for h2o: filename=h2o-3.32.0.4-py2.py3-none-any.whl size=164670979 sha256=af0dda799a5dbff9da6a99761eea9ea59a19eeff015afa1189d197b3c1cf6b02
  Stored in directory: /root/.cache/pip/wheels/65/f4/0d/c9bb958d70c2e015c968cb91cbd7f1b486933056d422337d75
Successfully built h2o


In [3]:
import pandas as pd
import numpy as np 
import spacy 

from spacy.lang.en.stop_words import STOP_WORDS as stopwords
import re
from textblob import TextBlob

from wordcloud import WordCloud 
import matplotlib.pyplot as plt 
%matplotlib inline

import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix

import h2o
from h2o.automl import H2OAutoML
h2o.init()



Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp4j5d0smw
  JVM stdout: /tmp/tmp4j5d0smw/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp4j5d0smw/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_unknownUser_ea4otr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Loading Data

In [5]:
data = pd.read_excel('/content/drive/MyDrive/DataSets/QLC /Data1.xlsx',nrows=50000)
data = data.dropna()
data.head()

,Unnamed: 0,OpportunityId,OpportunityStageName,ActivityDescription
0,0,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#ProformaInvoice\r\nDear Sir,\r\n\r\nThanks fo..."
1,1,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#Emailed\r\nDear Sir,\r\n\r\nPlease find revis..."
2,2,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
3,3,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
4,4,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,#Follow up\r\n\r\nCalled on landline number\r\...


Converting it to Dictionary

In [6]:
#convert to dictionary
cov_dic = {'OpportunityId': str,
           'OpportunityStageName': str,
           'ActivityDescription': str}
df= data.astype(cov_dic)
df=df.drop(columns=['Unnamed: 0'])
df.head()

,OpportunityId,OpportunityStageName,ActivityDescription
0,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#ProformaInvoice\r\nDear Sir,\r\n\r\nThanks fo..."
1,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#Emailed\r\nDear Sir,\r\n\r\nPlease find revis..."
2,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
3,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
4,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,#Follow up\r\n\r\nCalled on landline number\r\...


Upper case Count

In [7]:
df['upper_count'] = df['ActivityDescription'].apply(lambda x: len([t for t in x.split() if t.isupper()]))

In [8]:
# Lower case
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x:str(x).lower())

#removing '-' in id
df['OpportunityId'] = df['OpportunityId'].apply(lambda x: str(x).replace('-', ''))

#removing '/r' and '/n
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: str(x).replace('\r', ' '))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: str(x).replace('\r\n', ' '))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: str(x).replace('\n', ' '))

#count and remove URLS
df['URLs_count'] = df['ActivityDescription'].apply(lambda x: len(re.findall(r'(http|https|ftp|ssh|www)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^+%&/~+#-])?',str(x))))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub(r'(http|https|ftp|ssh|www)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^+%&/~+#-])?','',str(x)))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub('(?:\s)www[^, ]*', '', str(x)))

#remove single letters
df['ActivityDescription'] = df.ActivityDescription.str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
# remove all numbers
df['ActivityDescription'] = df.ActivityDescription.str.replace('\d+', '')

df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub(r'[^\w ]+',"",x)) #removing special characters
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: ' '.join(x.split())) #removing double spaces

#counting hashtags
df['hash_count'] = df['ActivityDescription'].apply(lambda x : re.findall(r'(#[Aa])', x))
df['hash_count'] =df['hash_count'].apply(lambda x : len(x))

#count email
df['e_count'] = df['ActivityDescription'].apply(lambda x : re.findall(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+",x))
df['e_count'] = df['e_count'].apply(lambda x:len(x))
#remove email 
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub("[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+","",str(x)))

#stop word count
df['sw_count']= df['ActivityDescription'].apply(lambda x: len([t for t in x.split() if t in stopwords]))
# removing stop words
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x : ' '.join([t for t in x.split() if t not in stopwords]))

In [9]:
df['URLs_count'].value_counts()

0    49547
2      149
1      105
4       82
3       73
7        3
8        2
Name: URLs_count, dtype: int64

In [10]:
#removing top 20 frequet words
text = ' '.join(df['ActivityDescription'])
text = text.split()
freq_comm = pd.Series(text).value_counts()
f20 = freq_comm[:20]
f20

df['ActivityDescription']= df['ActivityDescription'].apply(lambda x : ' '.join([t for t in x.split() if t not in f20]))


In [11]:
#Label encoding target variable
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['OpportunityStageName']= label_encoder.fit_transform(df['OpportunityStageName']) 

In [12]:
#tokenization
from nltk.tokenize import sent_tokenize, word_tokenize 
#df['ActivityDescription']= df['ActivityDescription'].apply(lambda x: [word for word in word_tokenize(x)])
#word_tokenize(text)
df['ActivityDescription']= df['ActivityDescription'].apply(word_tokenize)

In [17]:
stemmer = nltk.stem.SnowballStemmer('english')
def tokenize(text):
   tokens = text
   stems = [stemmer.stem(item) for item in tokens if (item not in stopwords)]
   return stems

In [18]:
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x : tokenize(x))

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df, test_size=0.2, random_state=111)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words=None, max_df=0.75, max_features=1000, lowercase=False, ngram_range=(1,2))
train_vectors = vectorizer_tf.fit_transform(X_train.ActivityDescription)

In [21]:
test_vectors = vectorizer_tf.transform(X_test.ActivityDescription)

In [22]:
train_vectors

<39968x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 503665 stored elements in Compressed Sparse Row format>

In [23]:
vectorizer_tf.get_feature_names

<bound method CountVectorizer.get_feature_names of TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=0.75,
                max_features=1000, min_df=1, ngram_range=(1, 2), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words=None,
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x7f47a2981ef0>, use_idf=True,
                vocabulary=None)>

In [24]:
X_train.head()

,OpportunityId,OpportunityStageName,ActivityDescription,upper_count,URLs_count,hash_count,e_count,sw_count
45307,393AC61F59394E8BB75B04A25ADCBAD4,0,"[shanki, respons, drop, msg]",1,0,0,0,7
3558,04B567D3854046C18B0950E86DA6E1DD,8,"[pme, visit, visit, met, sudarshan, discuss, c...",1,0,0,0,8
6149,08344F755506469DA1EB6A25224014C3,2,"[firstcontactestablish, receiv, login, detail,...",0,0,0,0,6
12090,0F3935C85BF94798886834E07C3A8EE2,0,"[respond, call]",0,0,0,0,4
5487,07568ED723C34C5BA986CD1D04C1AB16,3,"[word, vishwamurthi, contact, aditya]",0,0,0,0,4


In [25]:
train_df=pd.DataFrame(train_vectors.toarray(), columns=vectorizer_tf.get_feature_names())
train_df=pd.concat([train_df,X_train['OpportunityStageName'].reset_index(drop=True)], axis=1)
train_df=pd.concat([train_df,X_train[['upper_count','hash_count','e_count','sw_count']].reset_index(drop=True)], axis=1)
train_df

,aaacqn,aaacqn id,aaacqn servic,abhishek,abl,accept,access,accord,account,account branch,account ifsc,account quantumlink,account run,accountsreceivableqlcin,accountsreceivableqlcin note,activ,ad,ad contact,ad slam,add,addit,addit year,address,address sandoz,address st,admin,advanc,advanc send,advic,agr,ahead,akshada,akshada gaikwad,ambassador,ambassador hotelmodel,amc,amc plan,amc pme,amc renew,amit,...,visit support,visit visit,wa,wait,want,web,websit,week,went,went meet,weâr,weâr happi,weâv,weâv help,whatsapp,wo,womum,wor,wor mumbai,word,work,work womum,write,wrote,year,year contract,year free,year rs,year year,yes,yesterday,youtub,youtub channel,yr,yrs,OpportunityStageName,upper_count,hash_count,e_count,sw_count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,7
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.487448,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,1,0,0,8
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,0,6
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.735139,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0,0,0,12
39964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,0,0,0,0
39965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2,0,0,10
39966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000

In [26]:
test_df=pd.DataFrame(test_vectors.toarray(), columns=vectorizer_tf.get_feature_names())
test_df=pd.concat([test_df,X_test['OpportunityStageName'].reset_index(drop=True)], axis=1)
test_df=pd.concat([test_df,X_test[['upper_count','hash_count','e_count','sw_count']].reset_index(drop=True)], axis=1)
test_df

,aaacqn,aaacqn id,aaacqn servic,abhishek,abl,accept,access,accord,account,account branch,account ifsc,account quantumlink,account run,accountsreceivableqlcin,accountsreceivableqlcin note,activ,ad,ad contact,ad slam,add,addit,addit year,address,address sandoz,address st,admin,advanc,advanc send,advic,agr,ahead,akshada,akshada gaikwad,ambassador,ambassador hotelmodel,amc,amc plan,amc pme,amc renew,amit,...,visit support,visit visit,wa,wait,want,web,websit,week,went,went meet,weâr,weâr happi,weâv,weâv help,whatsapp,wo,womum,wor,wor mumbai,word,work,work womum,write,wrote,year,year contract,year free,year rs,year year,yes,yesterday,youtub,youtub channel,yr,yrs,OpportunityStageName,upper_count,hash_count,e_count,sw_count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,9,0,0,0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.360888,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,9,2,0,0,6
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.430428,0.0,0.0,0.0,0.0,0.0,3,3,0,0,21
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
9989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087519,0.0,0.126784,0.0,0.10757,0.094654,0.095505,0.0,0.0,0.228195,0.10668,0.0,0.0,0.0,...,0.104722,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.082419,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,3,5,0,0,39
9990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000

In [27]:
h2o_train_df = h2o.H2OFrame(train_df)
h2o_test_df = h2o.H2OFrame(test_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
h2o_train_df.types

{'aaacqn': 'real',
 'aaacqn id': 'real',
 'aaacqn servic': 'real',
 'abhishek': 'real',
 'abl': 'real',
 'accept': 'real',
 'access': 'real',
 'accord': 'real',
 'account': 'real',
 'account branch': 'real',
 'account ifsc': 'real',
 'account quantumlink': 'real',
 'account run': 'real',
 'accountsreceivableqlcin': 'real',
 'accountsreceivableqlcin note': 'real',
 'activ': 'real',
 'ad': 'real',
 'ad contact': 'real',
 'ad slam': 'real',
 'add': 'real',
 'addit': 'real',
 'addit year': 'real',
 'address': 'real',
 'address sandoz': 'real',
 'address st': 'real',
 'admin': 'real',
 'advanc': 'real',
 'advanc send': 'real',
 'advic': 'real',
 'agr': 'real',
 'ahead': 'real',
 'akshada': 'real',
 'akshada gaikwad': 'real',
 'ambassador': 'real',
 'ambassador hotelmodel': 'real',
 'amc': 'real',
 'amc plan': 'real',
 'amc pme': 'real',
 'amc renew': 'real',
 'amit': 'real',
 'andheri': 'real',
 'andheri east': 'real',
 'annual': 'real',
 'annual mainten': 'real',
 'answer': 'real',
 'antis

In [29]:
h2o_train_df['OpportunityStageName'] = h2o_train_df['OpportunityStageName'].asfactor()
h2o_test_df['OpportunityStageName'] = h2o_test_df['OpportunityStageName'].asfactor()

In [30]:
aml = H2OAutoML(max_models = 10, seed = 10, exclude_algos = ["StackedEnsemble"], verbosity="info", nfolds=0, balance_classes=True, max_after_balance_size=0.3)

In [31]:
x=vectorizer_tf.get_feature_names()
y='OpportunityStageName'

In [ ]:
aml.train(x = x, y = y, training_frame = h2o_train_df, validation_frame=h2o_test_df)

AutoML progress: |
10:13:11.846: Project: AutoML_20210225_101311843
10:13:11.847: Cross-validation disabled by user: no fold column nor nfolds > 1.
10:13:11.853: Setting stopping tolerance adaptively based on the training frame: 0.005002001200800561
10:13:11.853: Build control seed: 10
10:13:11.853: Since cross-validation is disabled, and no leaderboard frame was provided, automatically split the training data into training and leaderboard frames in the ratio 90/10
10:13:15.588: training frame: Frame key: automl_training_py_1_sid_83bb    cols: 1005    rows: 35899  chunks: 41    size: 8747517  checksum: -9041490233139546984
10:13:15.591: validation frame: Frame key: py_2_sid_83bb    cols: 1005    rows: 9993  chunks: 11    size: 2370269  checksum: -5356998711068300942
10:13:15.902: leaderboard frame: Frame key: automl_leaderboard_py_1_sid_83bb    cols: 1005    rows: 4069  chunks: 41    size: 3849723  checksum: 1847102335573993059
10:13:15.902: blending frame: NULL
10:13:15.902: response 

In [ ]:
aml.leaderboard

In [ ]:
pred=aml.leader.predict(h2o_test_df)

In [ ]:

aml.leader.model_performance(h2o_test_df)

In [ ]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
out = h2o.get_model([mid for mid in model_ids if "XGBoost" in mid][0])
out

In [ ]:
out.convert_H2OXGBoostParams_2_XGBoostParams()

In [ ]:

xgb_clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, objective='multi:softprob', random_state=10, **{"updater": "grow_gpu"})

In [ ]:

xgb_clf.fit(train_vectors, X_train['OpportunityStageName'])

In [ ]:
predictions = xgb_clf.predict(test_vectors)

In [ ]:
cm = confusion_matrix(X_test['OpportunityStageName'], predictions)
print(cm)

In [ ]:
print('classification_report :\n',classification_report(X_test['OpportunityStageName'], predictions))

In [ ]:
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight('balanced',
                                             np.unique(X_train['OpportunityStageName']),
                                             X_train['OpportunityStageName']))

In [ ]:
class_weights

In [ ]:
weights = np.ones(X_train.shape[0], dtype = 'float')
weights

In [ ]:
for i, val in enumerate(X_train['OpportunityStageName']):
    weights[i] = class_weights[val]

weights

In [ ]:
xgb_clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, objective='multi:softprob', random_state=10, **{"updater": "grow_gpu"})

In [ ]:
xgb_clf.fit(train_vectors, X_train['OpportunityStageName'], sample_weight=weights)

In [ ]:
predictions = xgb_clf.predict(test_vectors)

In [ ]:
cm = confusion_matrix(X_test['OpportunityStageName'], predictions)
print(cm)

In [ ]:
print('classification_report :\n',classification_report(X_test['OpportunityStageName'], predictions))